In [2]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

['cleandata', 'jigsaw-unintended-bias-in-toxicity-classification']


In [3]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [4]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [5]:
x = baseTrain
y = labels

In [6]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TF = vectorizer_TF.fit_transform(x)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer()
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [9]:
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
#clf = svm.SVR()
#clf = linear_model.LinearRegression()
#clf = DecisionTreeRegressor(max_depth=10)
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')

#Regressão logistica
y_train_class = []
for t in range(len(y_train)):
    if(y_train[t]<0.5):
        y_train_class.append(0)
    else:
        y_train_class.append(1)
        
clf.fit(X_train,y_train_class)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
from sklearn import metrics
y_pred = clf.predict(X_test) 
y_pred_prob = clf.predict_proba(X_test) 

y_pred_class = y_pred
y_test_class = []
y_pred_prob_tox = []


for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
    y_pred_prob_tox.append(y_pred_prob[t][1])

mse = metrics.mean_squared_error(y_test, y_pred_prob_tox)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(acc)
print(prec)
print(rec)
print(mse)

with open('results_validation.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Logistic Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

0.7273441974555633
0.8300531373238784
0.7727803118701602
0.5086580086580087
0.04349015935165465


In [ ]:
# from sklearn import metrics
# y_pred = clf.predict(X_test) 

# y_pred_class = []
# y_test_class = []

# for t in range(len(y_test)):
#     if(y_test[t]<0.5):
#         y_test_class.append(0)
#     else:
#         y_test_class.append(1)
#     if(y_pred[t]<0.5):
#         y_pred_class.append(0)
#     else:
#         y_pred_class.append(1)

# mse = metrics.mean_squared_error(y_test, y_pred)
# auc = metrics.roc_auc_score(y_test_class, y_pred_class)
# acc = metrics.accuracy_score(y_test_class, y_pred_class)
# prec = metrics.precision_score(y_test_class, y_pred_class)
# rec = metrics.recall_score(y_test_class, y_pred_class)

# print(auc)
# print(mse)
# print(acc)
# print(prec)
# print(rec)

# with open('results_validation.csv', mode='w') as employee_file:
#     writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     writer.writerow(["Linear Regression"])
#     writer.writerow(["mse: ", mse])
#     writer.writerow(["auc: ", auc])
#     writer.writerow(["accuracy: ", acc])
#     writer.writerow(["precision: ", prec])
#     writer.writerow(["recall: ", rec])

In [12]:
y_pred_test = clf.predict_proba(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        print(ids[id])
        print(y_pred_test[id][1])
        writer.writerow([ids[id], y_pred_test[id][1]])

7000000
0.06919502382966954
7000001
0.06254674689284245
7000002
0.057210897836783306
7000003
0.02946783261675543
7000004
0.8775127831027446
7000005
0.033473606466348484
7000006
0.03253855932061869
7000007
0.06061043721698313
7000008
0.09289673787420553
7000009
0.04265196627441085
7000010
0.07151332413595014
7000011
0.32899960034301307
7000012
0.06391915065404472
7000013
0.02383735458917181
7000014
0.03556327842106994
7000015
0.04662606835921427
7000016
0.2088072111463099
7000017
0.05513581181686811
7000018
0.13737014655540344
7000019
0.07759978397919991
7000020
0.1295049004395059
7000021
0.10945416239097937
7000022
0.04320823404946161
7000023
0.4110469195567861
7000024
0.8264594888928142
7000025
0.07274881596242501
7000026
0.134616778554464
7000027
0.019392090208922744
7000028
0.1900432488491425
7000029
0.03229484740033392
7000030
0.040415968313520595
7000031
0.05122391230140652
7000032
0.6974099034630744
7000033
0.04612020003808142
7000034
0.2340598050456872
7000035
0.0655759309657374

7000953
0.09591204586198492
7000954
0.046741805608699734
7000955
0.08558401026250152
7000956
0.4822480078329343
7000957
0.046629788131404656
7000958
0.09692831165059979
7000959
0.10566036124003067
7000960
0.05888015909712373
7000961
0.03489564300525933
7000962
0.1873001182322309
7000963
0.053713285675660895
7000964
0.06177967936758906
7000965
0.22463579502804504
7000966
0.07772520603663673
7000967
0.10484524637320806
7000968
0.08857311096880638
7000969
0.11873362943345891
7000970
0.024497322773554143
7000971
0.04607380371196945
7000972
0.0982600289095876
7000973
0.0471890530684642
7000974
0.3656373284819447
7000975
0.20377911249972397
7000976
0.0825673303877203
7000977
0.14975203184760053
7000978
0.05193602579206043
7000979
0.06554679322481234
7000980
0.08215593391888351
7000981
0.02160149356261328
7000982
0.3543534220912918
7000983
0.02782202503450863
7000984
0.1738778564451041
7000985
0.019116596124630323
7000986
0.22418915210054344
7000987
0.1363729937194134
7000988
0.02675913799470

0.07100215572733445
7001869
0.08404081895081579
7001870
0.10064582722559734
7001871
0.02412596738034631
7001872
0.045838830104745674
7001873
0.05393817032953342
7001874
0.02971634143012047
7001875
0.09803153027271157
7001876
0.08564932967897973
7001877
0.03181065663613541
7001878
0.013428018296974079
7001879
0.07744268050270786
7001880
0.08014872967586284
7001881
0.045891884067297196
7001882
0.07027145469147188
7001883
0.07065691323262815
7001884
0.04145436246347047
7001885
0.09632421038594295
7001886
0.21721118426232192
7001887
0.03412152202339319
7001888
0.058482520113568665
7001889
0.09108622688902442
7001890
0.08666125519334154
7001891
0.12272695857522589
7001892
0.049931209565613026
7001893
0.0679699835586623
7001894
0.05805276960387372
7001895
0.04376247097730158
7001896
0.01748034873966653
7001897
0.04539259007211324
7001898
0.40070604806461013
7001899
0.12460962942636541
7001900
0.03662499388969757
7001901
0.02041954539735964
7001902
0.2150205163354368
7001903
0.053259777249294

0.035330610511634136
7002692
0.1048618943751165
7002693
0.04781086277492416
7002694
0.24096746481410874
7002695
0.13087571234176074
7002696
0.11959758451355465
7002697
0.0636524693370346
7002698
0.32513447522105277
7002699
0.06149565920152335
7002700
0.10975726688668153
7002701
0.0444564290396073
7002702
0.06939146196875531
7002703
0.06578175916518306
7002704
0.06686392079544311
7002705
0.04387944338722939
7002706
0.010220252338327481
7002707
0.03294672077757592
7002708
0.06630370140581665
7002709
0.0514059748658886
7002710
0.09355266275850775
7002711
0.07952251945533138
7002712
0.19414143442540369
7002713
0.09310532977949844
7002714
0.07544184357395571
7002715
0.07211666635514265
7002716
0.046767047873906994
7002717
0.05335523310365633
7002718
0.030514956975858046
7002719
0.07456350223225923
7002720
0.03959940145813453
7002721
0.033117515818744975
7002722
0.021865556246058567
7002723
0.019954008044810217
7002724
0.04050593576761888
7002725
0.03847895815203473
7002726
0.059876001539908

7003702
0.08183527054349368
7003703
0.09871592286363318
7003704
0.06826799280129298
7003705
0.0475749788784453
7003706
0.03063558890905122
7003707
0.11167570544352191
7003708
0.06890569609451477
7003709
0.2398619723162744
7003710
0.05037434125822474
7003711
0.07464660782619847
7003712
0.14436721022395854
7003713
0.07709685834548247
7003714
0.02248326972256274
7003715
0.06660427631839837
7003716
0.05698620314281769
7003717
0.0720097696170671
7003718
0.053575389903464304
7003719
0.04808228361696051
7003720
0.11444212692220734
7003721
0.06209920388507165
7003722
0.23013221165768172
7003723
0.06703558696724168
7003724
0.04107725357081491
7003725
0.08843009033243764
7003726
0.09304499608278126
7003727
0.15774394734903313
7003728
0.09730805850635417
7003729
0.03751106706381975
7003730
0.2631138299806339
7003731
0.09493558886680696
7003732
0.06424493786232247
7003733
0.026422029863922655
7003734
0.034443543451806335
7003735
0.05417231507431976
7003736
0.08543540257238665
7003737
0.05950122317

...

0.08774844576199575
7093837
0.03749963163961807
7093838
0.054274477341576186
7093839
0.28317394722017386
7093840
0.17039373969857824
7093841
0.7389791340024287
7093842
0.020760219320482977
7093843
0.03679602995873117
7093844
0.10708636608247608
7093845
0.058060469043449726
7093846
0.10175252069562993
7093847
0.06210750689637393
7093848
0.044105769229590586
7093849
0.07249357628160202
7093850
0.07251121419495911
7093851
0.08883197160954034
7093852
0.061725037762216106
7093853
0.1265181790166474
7093854
0.06917099048294688
7093855
0.11352655770889605
7093856
0.08167027695998233
7093857
0.07219339109116962
7093858
0.043346690712970454
7093859
0.3935006321628786
7093860
0.09182775296151158
7093861
0.05766609754397778
7093862
0.1153778258148748
7093863
0.06677134879164107
7093864
0.05009669556212444
7093865
0.054290905935697964
7093866
0.5338667334451803
7093867
0.07588026068681074
7093868
0.035398550130860885
7093869
0.05290510877370973
7093870
0.05968519544590748
7093871
0.069246451033266

0.056574090204491674
7094767
0.059575901842206104
7094768
0.040561021988684505
7094769
0.07614969706592677
7094770
0.24839488597033807
7094771
0.0422490015462349
7094772
0.05793778176219147
7094773
0.14138828754952162
7094774
0.3701526280659414
7094775
0.15523894108758066
7094776
0.028096609867174216
7094777
0.0785236291891302
7094778
0.06374941709509117
7094779
0.07044928580299652
7094780
0.06909513685519265
7094781
0.07507064009633523
7094782
0.06137232086875419
7094783
0.1405286414519819
7094784
0.06155175648905016
7094785
0.1482803145316192
7094786
0.05028282343425008
7094787
0.3228637905030679
7094788
0.05606929945703292
7094789
0.12339790107261865
7094790
0.08710186377919718
7094791
0.9959782714501854
7094792
0.287213545197815
7094793
0.044397037144625416
7094794
0.11334089210948295
7094795
0.046306380678027934
7094796
0.4274053878926374
7094797
0.06497999227516478
7094798
0.08432402546655068
7094799
0.029624484741388925
7094800
0.08432461119094244
7094801
0.02531536697417402
709

7095654
0.07778740729012569
7095655
0.018659028578487162
7095656
0.22422188793495743
7095657
0.080875997332235
7095658
0.10340568967368331
7095659
0.053299037401735166
7095660
0.06422166756013611
7095661
0.04050759790046231
7095662
0.6481160837556754
7095663
0.10388683565803916
7095664
0.13990922467106323
7095665
0.10829119880441362
7095666
0.03211990459100245
7095667
0.06309157637696967
7095668
0.9431420735003195
7095669
0.05175499332106918
7095670
0.4336758664055211
7095671
0.05059404422762339
7095672
0.04081714195929911
7095673
0.12706386718860505
7095674
0.03132275869125932
7095675
0.11099550187588803
7095676
0.07646498718461775
7095677
0.2590516526198701
7095678
0.08423998563128239
7095679
0.1318647576434102
7095680
0.18973770853673405
7095681
0.11921756636214101
7095682
0.0705004515321886
7095683
0.9985025965495617
7095684
0.8963554262073671
7095685
0.07171744100412945
7095686
0.034539693850467186
7095687
0.14229754091203772
7095688
0.059940777479387905
7095689
0.0568154865647616

7096645
0.0673750769015633
7096646
0.53117751150417
7096647
0.21360638844838772
7096648
0.025923069747661953
7096649
0.04536766468281112
7096650
0.4473646484065124
7096651
0.8869898480945643
7096652
0.052337784129138296
7096653
0.06592168147817101
7096654
0.05156534795671004
7096655
0.02823325395333387
7096656
0.23206044674179846
7096657
0.10436721251167136
7096658
0.1739774563356154
7096659
0.06650742420162911
7096660
0.4637677702942488
7096661
0.06139223461621558
7096662
0.05588432404893159
7096663
0.08068555809495999
7096664
0.007392458216757809
7096665
0.16853164020269917
7096666
0.11972172693926308
7096667
0.0922742082819572
7096668
0.2140291978625053
7096669
0.18737142104978677
7096670
0.05482977198632451
7096671
0.42453879602746525
7096672
0.08500461576339452
7096673
0.07946860329516385
7096674
0.04763620612862545
7096675
0.01557811414938141
7096676
0.20204257703948256
7096677
0.03586310715659582
7096678
0.04311590271438033
7096679
0.13544159899152627
7096680
0.04430525749823408

0.04967823896225902
7097244
0.053161636478310556
7097245
0.061786120645140316
7097246
0.9625292085965634
7097247
0.12618733883959493
7097248
0.24091438112508312
7097249
0.09424442592473238
7097250
0.9943392607949137
7097251
0.07898512234046494
7097252
0.10318141903892879
7097253
0.8896130077027572
7097254
0.1585016546903146
7097255
0.07522697448245608
7097256
0.3090001007357208
7097257
0.13032194796973343
7097258
0.07321906224321217
7097259
0.028567752826252502
7097260
0.16131282945761233
7097261
0.1073186174384796
7097262
0.03997791405438635
7097263
0.10954802269119855
7097264
0.061041549199423874
7097265
0.0885714038790292
7097266
0.03275480878732468
7097267
0.0709145936777542
7097268
0.02688200707947753
7097269
0.05583355531699939
7097270
0.07379977467515403
7097271
0.06834646331454958
7097272
0.06502279657090815
7097273
0.07544753248836168
7097274
0.7632225711817513
7097275
0.15597097427080858
7097276
0.31613485446971906
7097277
0.1361993504035732
7097278
0.574483584814776
7097279
